In [369]:
import os
import dotenv
from dotenv import load_dotenv
load_dotenv() 
import google.generativeai as genai

genai.configure(api_key="API")
import vertexai

from PIL import Image
import shutil

upload_file_cache = {}

In [ ]:
MODEL_ID="gemini-1.5-pro-latest"

model = genai.GenerativeModel(
    MODEL_ID,
    system_instruction=[
        "system prompt",
    ],
    generation_config=genai.GenerationConfig(
        max_output_tokens=1000,
        temperature=0.1,
    )
)

In [ ]:
from pathlib import Path

source_folder = Path('')
crops_directory = Path('')

for jpg_path in source_folder.glob("*.png"):
    img = Image.open(jpg_path)
    imgResize = img.resize((428*2, 240*2))
    imgResize.save(crops_directory / f"{jpg_path.stem}_frame.jpg", quality=90)

In [372]:
import hashlib

def upload_file(path):
    """upload file, but with caching!!!"""
    with open(path, "rb") as f:
        digest = hashlib.file_digest(f, "sha256").digest()

    if file_obj := upload_file_cache.get(digest, None):
        return file_obj

    response = genai.upload_file(path=str(path))
    upload_file_cache[digest] = response
    return response


In [ ]:
from tqdm import tqdm

uploaded_files = [
    upload_file(frame)
    for frame in tqdm(sorted(crops_directory.glob("*.jpg"))[::15])
]
    

In [ ]:
from google.generativeai.types import file_types

def find_evil_frames(frames: list[file_types.File], indent: int = 0) -> list[file_types.File]:
    print(f'{"  " * indent} {len(frames): 2}')
    if len(frames) == 0:
        return []
    
    if len(frames) == 1:
        response = model.generate_content(frames)
        if response._error:
            return frames
        else:
            return []
        
    else:   
        evil_frames = []
        for part in [
            frames[:len(frames)//2],
            frames[len(frames)//2:],
        ]:

            response = model.generate_content(part)
            if response._error:    # There is an evil frame in the first half of the frames. But which one?
                evil_frames += find_evil_frames(part, indent+1)

        return evil_frames

In [ ]:
evil_frames = find_evil_frames(uploaded_files)

In [ ]:
non_evil_frames = [
    f
    for f in uploaded_files
    if f not in evil_frames
]
len(non_evil_frames)

In [ ]:
# give us all evil frames
[
    frame.display_name
    for frame in evil_frames
]

In [378]:
frame_descriptions = [
    f"Frame {frame.display_name[:6]}:"
    for frame in non_evil_frames
]

In [ ]:
safety_settings=[
  {
    "category": "HARM_CATEGORY_DANGEROUS",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE",
  },
]

response = model.generate_content(
    [
        # An interleaved list of "Frame n:" and the corresponding image, for all non-evil frames
        *[
            x
            for xs in zip(frame_descriptions, non_evil_frames)
            for x in xs
        ], " user prompt"
  ],
    safety_settings=safety_settings,
)
print(response.text)